# Texte et distance de jaccard

On va dans ce TD regarder les proximités de fables de la Fontaine en comparant les mots les composants.

Ceci est une prémière approche dans l'analyse de textes. Des méthodes bien plus *sioux* sont utilisées en vrai (mettre les mots au singulier, les verbe à l'infinitif, etc)

## création de la distance

Pour créer une distance entre fable on devra :

1. récupérer les fables
2. extraire les mots qui les composent
3. créer une distance

### Récupération des fables

Les fables sont disponibles là : https://www.gutenberg.org/files/56327/56327-h/56327-h.htm

On va récupérer le fichier html puis entraire le texte d celui-ci

#### page internet

En utilisant la bibliothèque [requests](https://requests-fr.readthedocs.io/en/latest/)

In [ ]:
import requests

In [ ]:
page = requests.get("https://www.gutenberg.org/files/56327/56327-h/56327-h.htm")

#### parser la page

Il faut récuperer les tokens html de la page qui compoent les fables.

En regardant la page dans un browser et les outils de développement on voit que :
* les titres des fables sont des des balises `<h3>`
* le texte des fables est dans le `<div>` juste en dessous (un `div` *frère*)

Il existe plusieurs bibliothèque pour parser du html. On va utiliser [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Références** :
* https://www.dataquest.io/blog/web-scraping-tutorial-python/
* http://sametmax.com/parser-du-html-avec-beautifulsoup/

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
[type(item) for item in list(soup.children)]

On cherche tous les fables. En regardant dans le html on remarque que les `<h3>` semblent correspondre aux titres des fables. On vérifie :

In [ ]:
for x in soup.find_all('h3'):
    print(x)

Le contenu est dans le `div` juste après (c'est un frère) :

In [ ]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    print(poem)
    print("=====")

On doit maintenant récupérer le poème. Il est dans une balise `span` de class différente à `pagenum`.

**Attention** : le dernier poème n'a pas de contenu. On doit traiter ce cas particulier pour qu'il n'y ait pas d'erreurs.

In [ ]:
for x in soup.find_all('h3'):
    print(x.text)
    print('----')
    poem = x.find_next_sibling()
    contenu_div = poem.find('div')
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            print(y.text)
    print("====")

On peut maintenant créer nos fables :

In [ ]:
fables = {}

for x in soup.find_all('h3'):
    titre = x.text
    div_frere = x.find_next_sibling()
    contenu_div = div_frere.find('div')    
    if contenu_div is None: # le dernier poeme n'a pas de contenu
        continue
        
    contenu = ""
    for y in contenu_div.find_all('span'):
        if 'pagenum' not in y.attrs['class']:
            contenu += y.text + '\n'
    fables[titre] = contenu
    
fables

## extraire les mots des fables

Pour cela on va comencer par mettre le texte en minuscule puis utiliser une expression régulière pour chercher tous les mots.

**Références** :
* https://fr.wikipedia.org/wiki/Expression_régulière
* https://www.lucaswillems.com/fr/articles/25/tutoriel-pour-maitriser-les-expressions-regulieres



In [ ]:
import re

In [ ]:
bag_of_word = {}
for titre, texte in fables.items():
    bag_of_word[titre] = set(re.findall(r'\b\w+\b',texte.lower()))

In [ ]:
print(fables['LE RENARD ET LES POULETS D’INDE.'])

In [ ]:
bag_of_word['LE RENARD ET LES POULETS D’INDE.']

On supprime les mots de longueur pus petit ou égal à 2 pour essayer de ne conserver que les *vrais* mots.

In [ ]:
for titre in bag_of_word:
    new = set()
    for x in bag_of_word[titre]:
        if len(x) > 2:
            new.add(x)
    bag_of_word[titre] = new

### distance de jaccard

La  [distance de jaccard](https://fr.wikipedia.org/wiki/Indice_et_distance_de_Jaccard) est très utilisée pour comparer des objets décrit par des ensembles de propriétés. 

Elle vaut si :
* 1 : aucun mots en commun
* 0 : même vocabulaire

On commence par regarder les distance avec une fable de référence :

In [ ]:
base = bag_of_word['LE RENARD ET LES POULETS D’INDE.']

similarite = []
for titre, texte in bag_of_word.items(): 
    similarite.append([1 - len(base.intersection(texte)) / len(base.union(texte)), titre])
    
similarite.sort()

In [ ]:
similarite

On crée la matrice de distance :

In [ ]:
elements = list(bag_of_word.keys())

d = []

for x in elements:
    d.append([])
    base = bag_of_word[x]
    for y in elements:
        texte = bag_of_word[y]
        d[-1].append(1 - len(base.intersection(texte)) / len(base.union(texte)))
d

## Représentation des données

### MDS

Faites une MDS en 2 dimensions de ces données. Conclusion ?

### isomap

Faites une isomap en 2 dimensions de ces données en utilisant les 2 plus proches voisins. Chercher à augmenter le nombre de voisins. Conclusions ?